Food delivery dataset integration

Thier are diffrent fromats of dataset we are going o combine them and perform sql with help of pyhton and pandas

In [1]:
import pandas as pd
import sqlite3 # this is library is for sql queries

In [4]:
# lets load the data in notebook

orders_data = pd.read_csv('orders.csv')

orders_data.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


we can see that the oders dataset i loaded with coloumn
	order_id, user_id, 	restaurant_id	, order_date	, total_amount,	restaurant_name

In [5]:
orders_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


here orderdate is in object format we have to convert it into date format for safer side

In [8]:
orders_data["order_date"]=pd.to_datetime(orders_data["order_date"])

orders_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         10000 non-null  int64         
 1   user_id          10000 non-null  int64         
 2   restaurant_id    10000 non-null  int64         
 3   order_date       10000 non-null  datetime64[ns]
 4   total_amount     10000 non-null  float64       
 5   restaurant_name  10000 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 468.9+ KB


/tmp/ipython-input-1883302043.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  orders_data["order_date"]=pd.to_datetime(orders_data["order_date"])


In [9]:
orders_data[["order_id", "user_id", "restaurant_id"]].isnull().sum()


,0
order_id,0
user_id,0
restaurant_id,0


Here we can see thier is no null value so now
Lets move forward to check other dataset

In [10]:
users_data = pd.read_json('users.json')

users_data.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [7]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


In [13]:
#load restaurants.sql data
conn=sqlite3.connect('restaurants.db')
cursor=conn.cursor()

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

    cursor.executescript(sql_script)

In [15]:
restaurants_data=pd.read_sql_query("SELECT * FROM restaurants", conn)

restaurants_data.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [16]:
restaurants_data.isnull().sum()

,0
restaurant_id,0
restaurant_name,0
cuisine,0
rating,0


We can see here both resturant abdoders table have resturanst_name same column but diffrent value lets keeponly one that is from master data

In [17]:
orders_data=orders_data.drop(columns=["restaurant_name"])


In [18]:
orders_data.head()

,order_id,user_id,restaurant_id,order_date,total_amount
0,1,2508,450,2023-02-18,842.97
1,2,2693,309,2023-01-18,546.68
2,3,2084,107,2023-07-15,163.93
3,4,319,224,2023-10-04,1155.97
4,5,1064,293,2023-12-25,1321.91


In [19]:
order_users_data=pd.merge(
    orders_data,
    users_data,
    on="user_id",
    how="left"
    )

In [20]:
final_data=pd.merge(
    order_users_data,
    restaurants_data,
    on="restaurant_id",
    how="left"
)

Now lets check the final dataset

In [21]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         10000 non-null  int64         
 1   user_id          10000 non-null  int64         
 2   restaurant_id    10000 non-null  int64         
 3   order_date       10000 non-null  datetime64[ns]
 4   total_amount     10000 non-null  float64       
 5   name             10000 non-null  object        
 6   city             10000 non-null  object        
 7   membership       10000 non-null  object        
 8   restaurant_name  10000 non-null  object        
 9   cuisine          10000 non-null  object        
 10  rating           10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 859.5+ KB


In [22]:
final_data.head()

,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating
0,1,2508,450,2023-02-18,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [24]:
final_data.to_csv(" final_food_delivery_dataset.csv", index=False)

Here i validated all the datset first ,converted date column and removed redundant column resturant_name

#Data analysis

In [25]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         10000 non-null  int64         
 1   user_id          10000 non-null  int64         
 2   restaurant_id    10000 non-null  int64         
 3   order_date       10000 non-null  datetime64[ns]
 4   total_amount     10000 non-null  float64       
 5   name             10000 non-null  object        
 6   city             10000 non-null  object        
 7   membership       10000 non-null  object        
 8   restaurant_name  10000 non-null  object        
 9   cuisine          10000 non-null  object        
 10  rating           10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 859.5+ KB


In [28]:
final_data["order_month"]=final_data["order_date"].dt.to_period("M")

In [29]:
order_per_month=final_data.groupby("order_month")["total_amount"].sum()
order_per_month

,total_amount
order_month,
2023-01,645818.76
2023-02,630867.40
2023-03,716738.98
2023-04,666097.18
2023-05,668428.61
2023-06,610822.93
2023-07,688559.45
2023-08,694987.58
2023-09,653838.07


The revenue was stable in 2023 but we can see the its very low in 2024 may the due to limited data availability

User behavior patterns

In [30]:
#lets check Oders per user
orders_per_user=final_data.groupby("user_id").size()
orders_per_user.describe()

,0
count,2883.000000
mean,3.468609
std,1.792682
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,13.000000


City-wise Performance

In [33]:
#Oders by city
orders_by_city=final_data.groupby("city").size().sort_values(ascending=False)
orders_by_city

,0
city,
Bangalore,2751
Chennai,2469
Pune,2430
Hyderabad,2350


In [44]:
# revenue by city
orders_by_city=final_data.groupby("city")["total_amount"].sum().sort_values(ascending=False)
orders_by_city


,total_amount
city,
Bangalore,2206946.58
Chennai,1990513.03
Pune,1924797.93
Hyderabad,1889366.58


We see that revenue genarted by Bangalore also oders

Cuisine-wise Performance


In [36]:
#Orders by revenue
orders_by_cuisin = final_data.groupby("cuisine").size().sort_values(ascending=False)
orders_by_cuisin

,0
cuisine,
Mexican,2581
Italian,2532
Indian,2469
Chinese,2418


In [39]:
# Revenue by Cuisine
revenue_by_cuisin = final_data.groupby("cuisine")["total_amount"].sum().sort_values(ascending=False)
revenue_by_cuisin

,total_amount
cuisine,
Mexican,2085503.09
Italian,2024203.80
Indian,1971412.58
Chinese,1930504.65


Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [53]:
#lets calcultae distinct restuatrant per cuisine
restaurants_per_cuisine = (
    final_data
    .groupby("cuisine")["restaurant_id"]
    .nunique()
    .sort_values()
)

restaurants_per_cuisine


,restaurant_id
cuisine,
Chinese,120
Indian,126
Italian,126
Mexican,128


In [55]:
# lets combine revenue_by_cuisin and restaurants_per_cuisine
cuisine_summary = pd.DataFrame({
    "distinct_restaurants": restaurants_per_cuisine,
    "total_revenue": revenue_by_cuisin
})

cuisine_summary


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


Membership Impact (Gold vs Regular)

In [41]:
orders_by_membership = final_data.groupby("membership").size()
orders_by_membership

,0
membership,
Gold,4987
Regular,5013


In [42]:
avg_orders_by_membership = final_data.groupby("membership")["total_amount"].mean()
avg_orders_by_membership

,total_amount
membership,
Gold,797.145556
Regular,805.158434


# Solutions for the questions

COMMON PRECOMPUTATIONS

Gold Members Filter

In [59]:
gold_data=final_data[final_data["membership"]=="Gold"]

Gold Members – City Summary

In [61]:
gold_city_summary=gold_data.groupby("city").agg(
    total_orders=("order_id", "count"),
    total_revenue=("total_amount", "sum"),
    avg_order_value=("total_amount", "mean")
    )
gold_city_summary

,total_orders,total_revenue,avg_order_value
city,,,
Bangalore,1254,994702.59,793.223756
Chennai,1337,1080909.79,808.459080
Hyderabad,1112,896740.19,806.421034
Pune,1284,1003012.32,781.162243


City-Level Summary

In [60]:
city_summary=final_data.groupby("city").agg(
    total_orders=("order_id", "count"),
    total_revenue=("total_amount", "sum"),
    avg_order_value=("total_amount", "mean")
    )
city_summary

,total_orders,total_revenue,avg_order_value
city,,,
Bangalore,2751,2206946.58,802.234308
Chennai,2469,1990513.03,806.202118
Hyderabad,2350,1889366.58,803.985779
Pune,2430,1924797.93,792.097914


Restaurant-Level Summary

In [90]:
restuarnt_level_summary=final_data.groupby("restaurant_id").agg(
    total_orders=("order_id", "count"),
    total_revenue=("total_amount", "sum"),
    avg_order_value=("total_amount", "mean")
    )

User-Level Summary

In [71]:
user_level_summary=final_data.groupby("user_id").agg(
    total_orders=("order_id", "count"),
    total_spend=("total_amount", "sum"),
    avg_order_value=("total_amount", "mean")
    )

Cuisine-Level Summary

In [64]:
cuisine_summary=final_data.groupby("cuisine").agg(
    total_orders=("order_id", "count"),
    total_revenue=("total_amount", "sum"),
    distinct_restaurants=("restaurant_id", "nunique"))



1)Which city has the highest total revenue (total_amount) from Gold members?  

In [66]:
gold_city_summary["total_revenue"].idxmax()

'Chennai'

In [67]:
gold_city_summary.sort_values(by="total_revenue", ascending=False)

,total_orders,total_revenue,avg_order_value
city,,,
Chennai,1337,1080909.79,808.459080
Pune,1284,1003012.32,781.162243
Bangalore,1254,994702.59,793.223756
Hyderabad,1112,896740.19,806.421034


Which cuisine has the highest average order value across all orders?
Indian

Chinese

Italian

Mexican

In [68]:
cuisine_summary["avg_oder_value"]=(
    cuisine_summary["total_revenue"]/cuisine_summary["total_orders"]
)

In [69]:
cuisine_summary["avg_oder_value"].idxmax()

'Mexican'

How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

< 500

500 – 1000

1000 – 2000

 > 2000

In [72]:
high_value=user_level_summary[user_level_summary["total_spend"]>1000].shape[0]
high_value

2544

Which restaurant rating range generated the highest total revenue?

3.0 – 3.5

3.6 – 4.0

4.1 – 4.5

4.6 – 5.0

In [79]:
#lets create rating range column
rating_bins = [3.0,3.5,4.0,4.5,5.0]
rating_labels = ["3.0 – 3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"]

final_data["rating_range"] = pd.cut(
    final_data["rating"],
    bins=rating_bins,
    labels=rating_labels,
    include_lowest=True)

In [80]:
# revenue by rating
rating_level_summary = final_data.groupby("rating_range")["total_amount"].sum()
rating_level_summary

/tmp/ipython-input-177373744.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_level_summary = final_data.groupby("rating_range")["total_amount"].sum()


,total_amount
rating_range,
3.0 – 3.5,2136772.70
3.6-4.0,1717494.41
4.1-4.5,1960326.26
4.6-5.0,2197030.75


In [81]:
rating_level_summary.idxmax()

'4.6-5.0'

Among Gold members, which city has the highest average order value?

Hyderabad

Bangalore

Chennai

Pune

In [82]:
gold_city_summary["avg_order_value"].idxmax()

'Chennai'

Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

Indian

Chinese

Italian

Mexican


In [85]:
cuisine_summary.sort_values(by=["distinct_restaurants","total_revenue"],
                            ascending=[True,False]).head(1)

,total_orders,total_revenue,distinct_restaurants,avg_oder_value
cuisine,,,,
Chinese,2418,1930504.65,120,798.38902


What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

40%

45%

50%

55%

In [86]:
round((gold_data.shape[0]/final_data.shape[0])*100)

50

Which restaurant has the highest average order value but less than 20 total orders?

Grand Cafe Punjabi

Grand Restaurant South Indian

Ruchi Mess Multicuisine

Ruchi Foods Chinese

In [91]:
restuarnt_level_summary[
    restuarnt_level_summary["total_orders"]<20].sort_values(by="avg_order_value", ascending=False).head(1)

,total_orders,total_revenue,avg_order_value
restaurant_id,,,
294,13,13522.89,1040.222308


Which combination contributes the highest revenue?

Gold + Indian cuisine

Gold + Italian cuisine

Regular + Indian cuisine

Regular + Chinese cuisine

In [92]:


final_data.groupby(["membership","cuisine"])["total_amount"].sum().sort_values(ascending=False).head(1)

,,total_amount
membership,cuisine,
Regular,Mexican,1072943.3


During which quarter of the year is the total revenue highest?

Q1 (Jan–Mar)

Q2 (Apr–Jun)

Q3 (Jul–Sep)

Q4 (Oct–Dec)

In [98]:
final_data["order_quarter"]=final_data["order_date"].dt.to_period("Q")
final_data.groupby("order_quarter")["total_amount"].sum().idxmax()

Period('2023Q3', 'Q-DEC')

How many total orders were placed by users with Gold membership?


In [99]:
gold_data.shape[0]

4987

What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [100]:
round(city_summary.loc["Hyderabad","total_revenue"])

1889367

How many distinct users placed at least one order?


In [101]:
final_data["user_id"].nunique()

2883

What is the average order value (rounded to 2 decimals) for Gold members?

In [102]:
round(gold_data["total_amount"].mean(),2)

np.float64(797.15)

How many orders were placed for restaurants with rating ≥ 4.5?

In [104]:
high_rate=final_data[final_data["rating"]>=4.5].shape[0]
high_rate

3374

How many orders were placed in the top revenue city among Gold members only

In [105]:
gold_city_summary.loc[gold_city_summary["total_revenue"].idxmax(),"total_orders"]

np.int64(1337)

The total number of rows in the final merged dataset is

In [106]:
final_data.shape[0]

10000